In [1]:
import lxml
import cchardet
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import requests, re, html

In [84]:
df_big = pd.DataFrame(columns=['ID','Title','IngredientsRaw','TotalTime','NumSteps', 
                           'AvgRating','NumReviews', 'RecipeURL'])

This script can scrape about 2-3k recipes until the server connection resets.    I ran it several times, each time starting at the recipe the previous run timed out.  A few iterations of this got me about 16k recipes, enough to train a model.

In [1]:
# %%time

def normalize_string(string):
    # Convert all named and numeric character references (e.g. &gt;, &#62;)
    unescaped_string = html.unescape(string)
    return re.sub(
        r"\s+", " ",
        unescaped_string.replace("\xa0", " ")
        .replace("\n", " ")  # &nbsp;
        .replace("\t", " ").strip(),)

def getrecipe(url, retry_count=0):
    sleep(0.01 + 0.02*np.random.random())
    
    df = pd.DataFrame(columns=['ID','Title','IngredientsRaw','TotalTime','NumSteps', 
                           'AvgRating','NumReviews', 'RecipeURL'])
    df.loc[0,'RecipeURL'] = url
    
    try: 
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
    except ConectionResetError as e:
        if retry_count == 5:
            raise e
        time.sleep(1 + 2*np.random.random())
        getrecipe(url, retry_count + 1)        
    
    if soup.find("h1").get_text() != 'Whoops...':
        
        df.loc[0,'Title'] =  soup.find("h1").get_text()
            
        try:
            NumReviews =  soup.find("a", {"class": "reviews-count__link theme-color"}).getText()
            df.loc[0,'NumReviews'] = eval(NumReviews)
        except:
            pass


        try:
            ingredients = soup.findAll("li", {"class": "recipe-ingredients__item"})
            df.loc[0,'IngredientsRaw'] = [normalize_string(ingredient.get_text()) + '#item' 
                                          for ingredient in ingredients]

        except:
            pass

        try:
             df.loc[0,'TotalTime'] = (soup.find("div", {"class": "recipe-facts__time"})
                                         .get_text().replace('READY IN: ',''))
        except:
            pass

        try:
            instructions = soup.findAll("li", {"class": "recipe-directions__step"})
            df.loc[0,'NumSteps'] = len(instructions)
        except:
            pass

        try:
            AvgRating = soup.find('div',class_='stars-rate__filler').get("style")[:-2].split(':')[1]
            df.loc[0,'AvgRating'] = float(AvgRating)/20
        except:
            pass
        
    return df
    
for i in range(23561,40000):
    url  = 'https://www.food.com/recipe/' + str(i) 
    df = getrecipe(url,5)
    df.loc[0,'ID'] = i
    df_big = df_big.append(df)
    

In [93]:
df_bigall = pd.concat([df_big1, df_big2, df_big3, df_big4])
df_bigall

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,10000,Tomato Paste,"[48 large tomatoes#item, 2 teaspoons salt#item]",3hrs 15mins,8,3.0,2,https://www.food.com/recipe/10000
0,10001,Aloo Dum,"[1⁄2 kg potato (small and round)#item, oil (fo...",4mins,18,NaN,NaN,https://www.food.com/recipe/10001
0,10002,Caulinana salad,"[1 small cauliflower#item, 2 bananas#item, 2 t...",15mins,11,NaN,NaN,https://www.food.com/recipe/10002
0,10003,Spicy Corn Salad With Avocado Dressing,"[4 cups whole kernel corn (I use frozen)#item,...",12mins,10,3.67,3,https://www.food.com/recipe/10003
0,10004,Ginger Beer,"[TO MAKE THE GINGER BEER#item, 1 1⁄2 teaspoons...",312hrs 5mins,18,4.0,2,https://www.food.com/recipe/10004
...,...,...,...,...,...,...,...,...
0,26281,Steak Sauce Extraordinaire,[1 - 1 1⁄2 lb fresh rhubarb or 1 -1 1/2 lb fro...,1hr 20mins,4,5.0,2,https://www.food.com/recipe/26281
0,26282,Beef Barbecue Biscuits,"[3⁄4 lb ground beef#item, 1⁄2 cup barbecue sau...",25mins,4,3.0,1,https://www.food.com/recipe/26282
0,26283,A-1 Dumplings,"[1 cup flour#item, 1⁄2 teaspoon salt#item, 2 t...",33mins,6,5.0,7,https://www.food.com/recipe/26283
0,26284,Quick & Easy Fried Rice,"[beef or shrimp, sliced thinly (whatever you h...",20mins,17,4.56,9,https://www.food.com/recipe/26284


In [137]:
test = df_bigall.set_index('ID').sort_index()
test['IngredientsRaw'] = test['IngredientsRaw'].apply(lambda x: ', '.join(x) if np.all(pd.notnull(x)) else x)
test.to_csv('data/df26285str.csv',index=True)
df26285str = pd.read_csv("data/df26285str.csv")
df26285str

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,10000,Tomato Paste,"48 large tomatoes#item, 2 teaspoons salt#item",3hrs 15mins,8.0,3.00,2.0,https://www.food.com/recipe/10000
1,10001,Aloo Dum,"1⁄2 kg potato (small and round)#item, oil (for...",4mins,18.0,NaN,NaN,https://www.food.com/recipe/10001
2,10002,Caulinana salad,"1 small cauliflower#item, 2 bananas#item, 2 ta...",15mins,11.0,NaN,NaN,https://www.food.com/recipe/10002
3,10003,Spicy Corn Salad With Avocado Dressing,"4 cups whole kernel corn (I use frozen)#item, ...",12mins,10.0,3.67,3.0,https://www.food.com/recipe/10003
4,10004,Ginger Beer,"TO MAKE THE GINGER BEER#item, 1 1⁄2 teaspoons ...",312hrs 5mins,18.0,4.00,2.0,https://www.food.com/recipe/10004
...,...,...,...,...,...,...,...,...
16281,26281,Steak Sauce Extraordinaire,1 - 1 1⁄2 lb fresh rhubarb or 1 -1 1/2 lb froz...,1hr 20mins,4.0,5.00,2.0,https://www.food.com/recipe/26281
16282,26282,Beef Barbecue Biscuits,"3⁄4 lb ground beef#item, 1⁄2 cup barbecue sauc...",25mins,4.0,3.00,1.0,https://www.food.com/recipe/26282
16283,26283,A-1 Dumplings,"1 cup flour#item, 1⁄2 teaspoon salt#item, 2 te...",33mins,6.0,5.00,7.0,https://www.food.com/recipe/26283
16284,26284,Quick & Easy Fried Rice,"beef or shrimp, sliced thinly (whatever you ha...",20mins,17.0,4.56,9.0,https://www.food.com/recipe/26284


In [139]:
test = df_bigall.set_index('ID').sort_index()
test.to_csv('data/df26285list.csv',index=True)
df26285list = pd.read_csv("data/df26285list.csv")
df26285list

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,10000,Tomato Paste,"['48 large tomatoes#item', '2 teaspoons salt#i...",3hrs 15mins,8.0,3.00,2.0,https://www.food.com/recipe/10000
1,10001,Aloo Dum,"['1⁄2 kg potato (small and round)#item', 'oil ...",4mins,18.0,NaN,NaN,https://www.food.com/recipe/10001
2,10002,Caulinana salad,"['1 small cauliflower#item', '2 bananas#item',...",15mins,11.0,NaN,NaN,https://www.food.com/recipe/10002
3,10003,Spicy Corn Salad With Avocado Dressing,['4 cups whole kernel corn (I use frozen)#item...,12mins,10.0,3.67,3.0,https://www.food.com/recipe/10003
4,10004,Ginger Beer,"['TO MAKE THE GINGER BEER#item', '1 1⁄2 teaspo...",312hrs 5mins,18.0,4.00,2.0,https://www.food.com/recipe/10004
...,...,...,...,...,...,...,...,...
16281,26281,Steak Sauce Extraordinaire,['1 - 1 1⁄2 lb fresh rhubarb or 1 -1 1/2 lb fr...,1hr 20mins,4.0,5.00,2.0,https://www.food.com/recipe/26281
16282,26282,Beef Barbecue Biscuits,"['3⁄4 lb ground beef#item', '1⁄2 cup barbecue ...",25mins,4.0,3.00,1.0,https://www.food.com/recipe/26282
16283,26283,A-1 Dumplings,"['1 cup flour#item', '1⁄2 teaspoon salt#item',...",33mins,6.0,5.00,7.0,https://www.food.com/recipe/26283
16284,26284,Quick & Easy Fried Rice,"['beef or shrimp, sliced thinly (whatever you ...",20mins,17.0,4.56,9.0,https://www.food.com/recipe/26284
